检查每个claim，第一个检索结果如果是 factcheck.afp.com 或者 reuters.com ，
然后再核对title是否对应的上claim：
找claim第一个单词，再title里找到对应的起始位置，然后终止位置为...
查看title是否在claim中出现，如果是，那么FALSE改为TRUE


In [1]:
import json
import sys
dirs = ["../..", "../create_prompt_llm"]
for _dir in dirs:
    if _dir not in sys.path:
        sys.path.append(_dir)
import prompt_rag

search_engine = "brave"


with open(f"/home/hanlv/workspace/data/machine_learning/dataset/research/misinformation_dataset/COVMIS-main/data/train_{search_engine}_search.json", "r") as f:
    data_search = json.load(f)
try:
    with open(f"/home/hanlv/workspace/data/machine_learning/dataset/research/misinformation_dataset/COVMIS-main/data/train_{search_engine}_search_llm.json", "r") as f:
        data_search_llm = json.load(f)
except:
    data_search_llm = [{
        "claim": i["claim"],
        "claimant": i["claimant"],
        "label": i["label"],
        "date": i["date"],
    } for i in data_search]
    with open(f"/home/hanlv/workspace/data/machine_learning/dataset/research/misinformation_dataset/COVMIS-main/data/train_{search_engine}_search_llm.json", "w") as f:
        json.dump(data_search_llm, f, indent=4)

def save_diff(x, model_name, diff_version):
    with open(f"output/diff_{model_name}_v{diff_version}.json", "w") as f:
        json.dump(x, f, indent=4)

def load_diff(model_name, diff_version):
    try:
        with open(f"output/diff_{model_name}_v{diff_version}.json", "r") as f:
            return json.load(f)
    except:
        data_init = [{"claim": i["claim"], "label": int(i["label"])} for i in data_search_llm]
        with open(f"output/diff_{model_name}_v{diff_version}.json", "w") as f:
            json.dump(data_init, f, indent=4)
        return data_init


In [2]:
"'how are you".split(' ')


["'how", 'are', 'you']

In [3]:
cnt = 0
for item in data_search:
    claim = item["claim"]
    title = item[f"{search_engine}_search_results"]["web"]["results"][0]["title"]
    url = item[f"{search_engine}_search_results"]["web"]["results"][0]["url"]
    check_list = ["poynter"] # 

    ok = False
    for site in check_list:
        if site in url:
            ok = True
            break
    if not ok:
        continue
    cnt += 1

    print(title, claim)
    # break
cnt


FALSE: Dr Mikovits, an established scientist, revealed that Dr ... Dr Mikovits, an established scientist, revealed that Dr Anthony Fauci was paid to cover up the extent of the problem caused by COVID-19. Also she revealed that SARS-CoV-2 was created in a lab and that influenza vaccines increase COVID-19 mortality by 36%.
FALSE: An official document approved by the Italian Health Ministry ... An official document approved by the Italian Health Ministry bans autopsies on COVID-19 deceased patients.
FALSE: Filipina actress Angel Locsin "tested positive" for the ... Filipina actress Angel Locsin tested positive for the coronavirus after conducting several relief operations.
FALSE: A woman says that in a video that everything is a hoax, ... A woman says that in a video that everything is a hoax, that the virus already existed and that there are even patents that prove it.
UV radiation is not a common way to kill viruses and bacteria in ... Ultraviolet Radiation is administered into the body

2909

In [29]:
cnt = 0
for item in data_search:
    claim = item["claim"]
    title = item[f"{search_engine}_search_results"]["web"]["results"][0]["title"]
    url = item[f"{search_engine}_search_results"]["web"]["results"][0]["url"]
    check_list = ["factcheck.afp.com", "reuters.com"] # Poynter

    ok = False
    for site in check_list:
        if site in url:
            ok = True
            break
    if not ok:
        continue

    # cnt += 1
    pos_st = title.find(claim.split(' ')[0])
    if pos_st == -1:
        continue
    pos_en = len(title)
    if title.endswith("..."):
        pos_en -= 4
    
    if title[pos_st:pos_en] in claim:
        cnt += 1
        # print(title + ';' + claim)
    
cnt


836